# Predicting Heart Disease: DSCI 100 Group Project Proposal (Group 23)

### Introduction:

Cardiovascular diseases (CVDs), or heart diseases, encompasses a range of medical conditions that affect the heart and blood vessels. Worldwide, CVDs are the leading cause of death with nearly 18 million deaths a year. CVDs  include coronary heart disease, strokes and TIAs, peripheral arterial disease, and aortic disease. Heart disease is often “silent” and is not diagnosed until a major event like a heart attack, stroke, or heart failure occurs. Early detection is key to treatment, prevention, and management of heart disease; our data analysis aims to help with this. Risk factors for heart disease include high blood pressure, high blood cholesterol, smoking, diabetes, obesity, family history, and increased age. 

The dataset used in our analysis is from the following repository: https://archive.ics.uci.edu/dataset/45/heart+disease. The data was published in 1988 and collected in Cleveland. The dataset consists of 303 observations and 14 variables: age, sex, chest pain type, resting blood pressure, serum cholesterol, fasting blood sugar, resting electrocardiogram results, maximum heart rate achieved, presence of exercise induced angina, ST depression induced by exercise relative to rest, the slope of the peak exercise ST segment, number of major vessels colored by fluoroscopy, thalassemia presence, and diagnosis of heart disease.

The question we aim to address is: can we predict if a patient has heart disease based on their age, resting blood pressure, serum cholesterol, and maximum resting heart rate?


### Preliminary exploratory data analysis:


- Demonstrate that the dataset can be read from the web into R 
- Clean and wrangle your data into a tidy format
- Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.


### Methods:

- Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
- Describe at least one way that you will visualize the results


### Expected outcomes and significance:


- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?

